In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100



In [18]:
#Getting Player Info Files

player_info_england = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
player_info_italy = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
player_info_spain = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
player_info_france = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')

In [8]:
print(player_info_england.columns)
print(player_info_italy.columns)

Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')
Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')


In [12]:
player_info_df = pd.concat([player_info_england, player_info_italy])

In [16]:
player_info_df = pd.concat([player_info_df, player_info_spain])

In [19]:
player_info_df = pd.concat([player_info_df, player_info_france])

In [24]:
# Cleaning repeated players

player_info_df_new = player_info_df.drop_duplicates()
print(len(player_info_df))
print(len(player_info_df_new))

29509
9115


In [31]:
#Loading Player Injuries

player_injuries_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')
len(player_injuries_df)

55216

In [87]:
# Merging Player Profile with Player Injuries

player_profile_injury_df = pd.DataFrame([])

player_profile_injury_df = pd.merge(left=player_injuries_df, right=player_info_df_new, left_on='name', right_on='PlayerName', how='left')
len(player_profile_injury_df)

56299

In [89]:
# Filtering NaN's 

player_profile_injury_df_filtered = player_profile_injury_df.dropna(subset=['PlayerName'])
len(player_profile_injury_df_filtered)

33974

In [132]:
player_info_df_new[player_info_df_new['Birth'] == 'April 22, 1987']
# player_info_df_new[player_info_df_new['PlayerFullName'] == 'John Mikel']


,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram
233,c0e27d1a,David Luiz,David Luiz Moreira Marinho,Brazil,https://fbref.com/req/202005121/images/headsho...,"April 22, 1987",189.0,84.0,DF-MF (CB-DM),RIGHT,78.0,4x Domestic League Champion||2015-16 Coupe de ...,https://fbref.com/en/players/c0e27d1a/David-Luiz,https://twitter.com/DavidLuiz_4,https://instagram.com/davidluiz_4/
3409,424d5f72,John Obi Mikel,John Michael Nchekube Obinna,Nigeria,https://fbref.com/req/202005121/images/headsho...,"April 22, 1987",189.0,86.0,MF (CM-DM),RIGHT,89.0,2x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/424d5f72/John-Obi...,NaN,https://instagram.com/john_mikel_obi/
6591,ddee7ba5,David Mateos,David Mateos Ramajo,Spain,NaN,"April 22, 1987",190.0,81.0,DF (CB),RIGHT,100.0,NaN,https://fbref.com/en/players/ddee7ba5/David-Ma...,NaN,NaN


In [123]:
# Determining missing players from PlayerName merge


player_profile_injury_df_missings = player_profile_injury_df[pd.isna(player_profile_injury_df['PlayerName'])]

player_profile_injury_df_missings = player_profile_injury_df_missings[['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId', 'name']]

player_profile_injury_df_missings = pd.merge(left=player_profile_injury_df_missings, right=player_info_df_new, left_on='name', right_on='PlayerFullName', how='left')
player_profile_injury_df_missings.head(50)
# player_profile_injury_df_missings = player_profile_injury_df_missings.dropna(subset=['PlayerFullName'])
# len(player_profile_injury_df_missings)

,Season,Injury,from,until,Days,Games missed,TMId,name,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12/13,Fractured Finger,"Feb 4, 2013","Feb 7, 2013",3 days,-,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10/11,Calf Problems,"Jul 19, 2010","Aug 12, 2010",24 days,1,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,09/10,Calf Problems,"Mar 1, 2010","Mar 25, 2010",24 days,5,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,08/09,Thigh Problems,"Nov 13, 2008","Nov 20, 2008",7 days,1,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,08/09,Thigh Problems,"Oct 13, 2008","Oct 23, 2008",10 days,2,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# Merging PlayerName merge with PlayerFullName merge

player_profile_injury_df_final = pd.concat([player_profile_injury_df_filtered, player_profile_injury_df_missings])
len(player_profile_injury_df_final)


34149

In [85]:
pd.DataFrame(player_profile_injury_df_final).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_profile_injury_df_final.csv', index=False)